In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# 数据准备
from sklearn.datasets import load_diabetes
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
diabetes = load_diabetes()
# 查看数据大小
print(diabetes.data.shape)
# 查看数据的keys
print(diabetes.keys())
# 打乱数据集
X,y = shuffle(diabetes.data,diabetes.target,random_state=13)
# 划分训练数据集和测试数据集
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=13)
# 将y转换为列向量
y_train,y_test = y_train.reshape(-1,1),y_test.reshape(-1,1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(442, 10)
dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])
(353, 10)
(89, 10)
(353, 1)
(89, 1)


In [7]:
# 定义线性回归模型主体
def linear_loss(X,y,w,b):
    """
    :param X: 特征
    :param y: 标签
    :param w: 权重
    :param b: 偏置
    :return: 损失函数
    """
    # 训练集的数量 -> (442,10)
    num_train = X.shape[0]
    # 特征数量
    num_feature = X.shape[1]
    # 线性回归预测值
    y_hat = np.dot(X,w) + b
    # 计算损失函数
    loss = np.sum((y_hat-y)**2)/num_train
    # 计算梯度
    dw = np.dot(X.T,(y_hat-y))/num_train # (10,442)*(442,1) -> (10,1) 对应的是矩阵形式的求导公式
    db = np.sum((y_hat-y))/num_train
    return y_hat,loss,db,dw

In [8]:
# 初始化参数
def initialize_parameters(dims):
    """
    :param dims: 特征数量
    :return: 权重和偏置
    """
    w = np.zeros((dims,1))
    b = 0
    return w,b

In [9]:
# 定义模型的训练过程
def linear_train(X,y,learning_rate,epochs=10000):
    """
    :param X: 特征
    :param y: 标签
    :param learning_rate: 学习率
    :param epochs: 迭代次数
    """
    loss_history = []
    # 初始化参数
    w,b=initialize_parameters(X.shape[1])

    # 迭代训练
    for i in range(epochs):
        # 计算当前迭代值，均方损失和梯度
        y_hat,loss,db,dw = linear_loss(X,y,w,b)
        # 更新参数
        w = w - learning_rate*dw
        b = b - learning_rate*db
        # 记录当前迭代损失
        loss_history.append(loss)
        # 打印当前迭代损失
        if i%1000==0:
            print("Epoch: {0}, loss: {1}".format(i,loss))
        # 记录模型参数
        params = {
            "w":w,
            "b":b
        }
        # 记录梯度信息
        grads = {
            "dw":dw,
            "db":db
        }
    return loss_history,params,grads

In [15]:
loss,params,grads = linear_train(X_train,y_train,learning_rate=0.1,epochs=3000000)

# print(params)

Epoch: 0, loss: 28947.98583569405
Epoch: 1000, loss: 3933.9101078040508
Epoch: 2000, loss: 3370.5924176385392
Epoch: 3000, loss: 3136.6312917808673
Epoch: 4000, loss: 3010.140075858654
Epoch: 5000, loss: 2933.626067682301
Epoch: 6000, loss: 2885.2418393733033
Epoch: 7000, loss: 2853.9334273143872
Epoch: 8000, loss: 2833.323945710775
Epoch: 9000, loss: 2819.5422998666722
Epoch: 10000, loss: 2810.1827352613873
Epoch: 11000, loss: 2803.727096625342
Epoch: 12000, loss: 2799.204803121045
Epoch: 13000, loss: 2795.9875813932117
Epoch: 14000, loss: 2793.6636658283555
Epoch: 15000, loss: 2791.959781125067
Epoch: 16000, loss: 2790.6922349710485
Epoch: 17000, loss: 2789.7359409745563
Epoch: 18000, loss: 2789.0045976458096
Epoch: 19000, loss: 2788.4378757502773
Epoch: 20000, loss: 2787.99304989936
Epoch: 21000, loss: 2787.6394746458777
Epoch: 22000, loss: 2787.354898215722
Epoch: 23000, loss: 2787.1229746987337
Epoch: 24000, loss: 2786.9315654835696
Epoch: 25000, loss: 2786.7715657082017
Epoch: 26

In [16]:
# 预测函数
def predict(X,params):
    """
    :param X: 特征
    :param params: 模型参数
    :return: 预测值
    """
    w = params["w"]
    b = params["b"]
    y_hat = np.dot(X,w) + b
    return y_hat

y_pred = predict(X_test,params)
print(y_pred)

[[240.56098094]
 [143.0648409 ]
 [184.83707272]
 [129.79223196]
 [ 72.85910028]
 [112.83072957]
 [119.46823845]
 [235.07118578]
 [204.64896109]
 [112.34607974]
 [156.41344365]
 [150.95773696]
 [114.56586322]
 [ 94.53298166]
 [191.85978665]
 [ 47.07641655]
 [ 88.42194755]
 [239.04548078]
 [188.2748663 ]
 [118.92604034]
 [199.10841765]
 [207.62753445]
 [163.14060076]
 [ 72.47291013]
 [128.34577552]
 [ 45.85433465]
 [196.45521047]
 [ 61.39516983]
 [233.36347789]
 [212.2194631 ]
 [197.77210528]
 [ 88.12048887]
 [215.94908704]
 [219.94633393]
 [174.11044262]
 [200.62518829]
 [ 48.2613713 ]
 [153.80992269]
 [175.63610004]
 [ 85.99844551]
 [214.37738132]
 [180.20699888]
 [170.64570642]
 [ 99.68037372]
 [ 67.43169589]
 [129.49629453]
 [192.84207013]
 [199.59413822]
 [217.13759305]
 [178.53416936]
 [201.80056856]
 [200.04960192]
 [120.27455834]
 [278.95931448]
 [164.53542786]
 [175.06286367]
 [198.71241294]
 [170.28764101]
 [180.16469317]
 [ 49.48161019]
 [ 81.90703977]
 [160.4555172 ]
 [140.74

回归模型$R^2$系数
衡量的是预测值对于真值的拟合好坏程度。通俗理解，假定$y_i$的方差为1个单位，则$R^2$表示使用该模型之后，$y_i$的残差的方差减少了多少"。比如R方等于0.8，则使用该模型之后残差的方差为原始$y_i$值方差的20%
$$
R^2 = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y_i})^2}{\sum_{i=1}^n (y_i - \bar{y})^2}
$$

In [19]:
# 计算回归模型R^2系数
def r2_score(y,y_hat):
    """
    :param y: 真实值
    :param y_hat: 预测值
    :return: R^2系数
    """
    y_mean = np.mean(y)
    # 总离差平方和
    ss_tot = np.sum((y-y_mean)**2)
    # 回归离差平方和
    ss_res = np.sum((y-y_hat)**2)
    # R^2系数
    r2 = 1 - ss_res/ss_tot
    return r2

print(r2_score(y_test,y_pred))

0.4315995097500155
